<img src="https://raw.githubusercontent.com/euroargodev/argopy/master/docs/_static/argopy_logo_long.png" alt="argopy logo" width="200"/> 

# Scatter map with data mode of one BGC variable

This notebook shows how to:
- download BGC-Argo index
- search for profiles with a specific parameter
- export search results as a Pandas DataFrame
- extract data mode values for this parameter
- download info from Argo NVS Reference Tables about a parameter
- plot a map where profile locations are color coded with parameter data mode

In [1]:
from argopy import ArgoIndex  #  This is the class to work with Argo index content
from argopy import ArgoNVSReferenceTables  # This is the class to retrieve data from Argo reference tables
from argopy.plot import scatter_map  # This is a function to easily make maps 

import numpy as np
from matplotlib import pyplot as plt

In [2]:
# Load an index of BGC profiles:
# idx = ArgoIndex(index_file='bgc-s').load()  # 'bgc-s' is a shortcut for 'argo_synthetic-profile_index.txt'
idx = ArgoIndex(index_file='bgc-b').load()  # 'bgc-b' is a shortcut for 'argo_bio-profile_index.txt'
idx

<argoindex.pyarrow>
Host: https://data-argo.ifremer.fr
Index: argo_bio-profile_index.txt.gz
Convention: argo_bio-profile_index (Bio-Profile directory file of the Argo GDAC)
In memory: True (363160 records)
Searched: False

In [3]:
# Define a parameter to work with:
# param = 'BBP470'
# param = 'DOXY'
param = 'BBP700'

In [5]:
# Get more verbose information about this parameter (usefull for plot titles):
reftbl = ArgoNVSReferenceTables().tbl('R03')
param_info = reftbl[reftbl['altLabel']==param].iloc[0]
param_info

altLabel                                                 BBP700
prefLabel             Particle backscattering at 700 nanometers
definition    Particle backscattering (/m) at 700 nm wavelen...
deprecated                                                false
id            http://vocab.nerc.ac.uk/collection/R03/current...
Name: 43, dtype: object

In [9]:
# Search for this parameter profiles:
idx.read_params(param)
idx

<argoindex.pyarrow>
Host: https://data-argo.ifremer.fr
Index: argo_bio-profile_index.txt.gz
Convention: argo_bio-profile_index (Bio-Profile directory file of the Argo GDAC)
In memory: True (363160 records)
Searched: False

In [8]:
# Possibly add a search in time (just to make a map more readable):
# idx.index = idx.search  # Trick to be able to chain multiple search methods with a single idx instance (new feature in dev to avoid this)
# idx.search_tim([-180,180,-90,90,'2023-01','2023-07'])
# idx

In [10]:
# Export search results to a dataframe:
df = idx.to_dataframe()
df.sample(5)

,file,date,latitude,longitude,ocean,profiler_code,institution_code,parameters,parameter_data_mode,date_update,wmo,cyc,institution,dac,profiler
49071,aoml/5904021/profiles/BD5904021_138.nc,2014-10-31 05:24:11,33.367,-127.976,P,846,AO,PRES TEMP_DOXY TPHASE_DOXY DOXY FLUORESCENCE_C...,RRRDRARARRD,2024-12-17 13:44:13,5904021,138,"AOML, USA",aoml,Teledyne Webb Research float with SBE conducti...
238431,coriolis/6903094/profiles/BD6903094_130.nc,2022-10-16 16:50:59,20.719,-108.388,P,834,IF,PRES C1PHASE_DOXY C2PHASE_DOXY TEMP_DOXY C1PHA...,RRRRRRRRRRARRRRRRRRRRRRDDDDRRRRRRRRRAAARRRRRRR...,2024-08-30 08:55:44,6903094,130,"Ifremer, France",coriolis,PROVOR V SBE
285575,csio/2902912/profiles/BR2902912_144.nc,2024-11-07 15:24:30,34.274,167.453,P,863,HZ,PRES PHASE_DELAY_DOXY TEMP_VOLTAGE_DOXY TEMP_D...,RRRRARARRRRRRR,2024-11-07 19:01:14,2902912,144,"CSIO, China Second Institute of Oceanography",csio,Navis-A float with SBE conductivity sensor
77490,aoml/5904857/profiles/BD5904857_097.nc,2019-09-02 22:24:58,-69.620,179.761,P,846,AO,PRES TEMP_DOXY TPHASE_DOXY DOXY FLUORESCENCE_C...,RRRDRAARARRRRRDRRD,2025-04-12 21:28:46,5904857,97,"AOML, USA",aoml,Teledyne Webb Research float with SBE conducti...
139649,coriolis/1901218/profiles/BD1901218_127.nc,2015-01-10 22:36:00,39.407,-32.429,A,841,IF,PRES MOLAR_DOXY DOXY,RRD,2020-08-03 21:23:28,1901218,127,"Ifremer, France",coriolis,PROVOR float with SBE conductivity sensor


In [14]:
# To make the scatter map, we need to have the data mode available in one DataFrame column
# so we need to add a new column with the DATA_MODE of the PARAMETER:
df["variables"] = df["parameters"].apply(lambda x: x.split())
df["%s_DM" % param] = df.apply(lambda x: x['parameter_data_mode'][x['variables'].index(param)] if param in x['variables'] else '', axis=1)

In [16]:
!pip install cartopy


Defaulting to user installation because normal site-packages is not writeable



[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [18]:
import cartopy.crs as ccrs

In [20]:
# Finally plot the map:
fig, ax = scatter_map(df,
                        hue="%s_DM" % param,
                        cmap="data_mode",
                        figsize=(10,6),
                        markersize=2,
                        markeredgecolor=None,
                        traj=False,  # Because some floats do weird things around 180/-180
                        set_global=False,
                        legend_title='%s data mode' % param)
ax.set_title("Data mode for '%s' (%s)\n%s profiles from the %s" % (param_info['prefLabel'], 
                                                                   param, 
                                                                   "{:,}".format(idx.N_MATCH), 
                                                                   idx.convention_title)
            );

C:\Users\Yadeesh\AppData\Roaming\Python\Python313\site-packages\argopy\utils\loggers.py:25: UserWarning: scatter_map requires matplotlib AND cartopy installed
  warnings.warn(msg)


NameError: name 'ccrs' is not defined

In [1]:
import pandas as pd
df=pd.read_csv("region.csv")
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 33840 entries, 0 to 33839
Data columns (total 18 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   CYCLE_NUMBER     33840 non-null  int64  
 1   DATA_MODE        33840 non-null  object 
 2   DIRECTION        33840 non-null  object 
 3   PLATFORM_NUMBER  33840 non-null  int64  
 4   POSITION_QC      33840 non-null  int64  
 5   PRES             33840 non-null  float64
 6   PRES_ERROR       23297 non-null  float64
 7   PRES_QC          33840 non-null  int64  
 8   PSAL             33840 non-null  float64
 9   PSAL_ERROR       23297 non-null  float64
 10  PSAL_QC          33840 non-null  int64  
 11  TEMP             33840 non-null  float64
 12  TEMP_ERROR       23297 non-null  float64
 13  TEMP_QC          33840 non-null  int64  
 14  TIME_QC          33840 non-null  int64  
 15  LATITUDE         33840 non-null  float64
 16  LONGITUDE        33840 non-null  float64
 17  TIME        